# Computing Sentiment analysis on each tweet
----
Using the Module "TextBlob", we firstly calculate the sentiment of a given text where the value ranges form value ranges from -1 (very negative) to +1 (very positive).

In [13]:
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import chardet   

# Function to calculate sentiment using TextBlob
def calculate_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

data = pd.read_csv('tweets_TRUMP_2016_2019.csv', encoding = "UTF-16")

# Convert 'created_at' to datetime
data['created_at'] = pd.to_datetime(data['created_at'], format='%m-%d-%Y %H:%M:%S')


# Calculate sentiment for each text
sentiments = []
for text in data['text']:
    sentiment = calculate_sentiment(text)
    sentiments.append(sentiment)

# Append the sentiment scores as a new column
data['sentiment'] = sentiments

data.head()


,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,sentiment
17788,Twitter for Android,Well the year has officially begun. I have man...,2016-01-01 23:00:09,2642,8495,False,683060169677344768,0.373611
17789,Twitter for Android,@sprinklermanus: @CNN @realDonaldTrump they're...,2016-01-01 21:29:56,933,3330,False,683037464504745985,0.800000
17790,Twitter for Android,@jallenaip: Hillary said she was in a Fog of W...,2016-01-01 06:08:06,2721,7490,False,682805477168779264,0.000000
17791,Twitter for iPhone,Happy New Year from #MarALago! Thank you to my...,2016-01-01 06:07:28,1948,8258,False,682805320217980929,0.590152
17792,Twitter for iPhone,#HappyNewYearAmerica! https://t.co/EeQb8PDrUe,2016-01-01 05:18:23,3434,9143,False,682792967736848385,0.000000
